In [1877]:
#******************************************************************************#
# Kaggle Titanic Competition                                                   #
# AXA_GerhardPachl - gerhard.pachl@axa-winterthur.ch                           #
# Version: 1                                                                   #
# first try - looking on the data, taking some variables(features)             #
# which seem to have an influence on survival or not like                      #
# Pclass, Sex, Age, Embarked                                                   #
#******************************************************************************#

In [1878]:
# initializing needed packages
import os
import pandas as pd
import numpy as np
import csv as csv
from sklearn.ensemble import RandomForestClassifier

In [1879]:
os.chdir('C:/Users/C924016/MyPythons/CSV/')
# importing the titanic train and test data set as a panda data frame
train_df = pd.read_csv('train.csv',header=0)
test_df  = pd.read_csv('test.csv',header=0)

In [1880]:
# first look on characteristics of the train and test data
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [1881]:
test_df.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [1882]:
# variable age has missing data - getting number of missing values 
test_df['Age'].isnull().values.ravel().sum()

86

In [1883]:
train_df['Age'].isnull().values.ravel().sum()

177

In [1884]:
# train['Age'].hist()
# P.show()

In [1885]:
# evaluate the other non numerical variables
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [1886]:
# take a look on Embarked - get unique values of variable Embarked --> boarding area
tr_boarding_area = list(enumerate(np.unique(train_df['Embarked']))) 
ts_boarding_area = list(enumerate(np.unique(test_df['Embarked'])))

In [1887]:
tr_boarding_area

[(0, nan), (1, 'C'), (2, 'Q'), (3, 'S')]

In [1888]:
ts_boarding_area

[(0, 'C'), (1, 'Q'), (2, 'S')]

In [1889]:
# get number of categories
train_df.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [1890]:
test_df.Embarked.value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64

In [1891]:
# getting number of missing values from age
train_df['Embarked'].isnull().values.ravel().sum()

2

In [1892]:
# getting number of missing values from age
test_df['Embarked'].isnull().values.ravel().sum()

0

In [1893]:
# change categorial values tonumeric and missing value of train data set 
# S == 1, C == 2, Q == 3, NAN like S (majority) == 1

In [1894]:
# change nan of train data to value 1
train_df.loc[ (train_df.Embarked.isnull()), 'Embarked'] = 'S'

In [1895]:
train_df['Embarked'].isnull().values.ravel().sum()

0

In [1896]:
# # S == 1, C == 2, Q == 3
train_df['Embarked'] = train_df['Embarked'].map( {'S': 1, 'C': 2, 'Q': 3} )
test_df['Embarked'] = test_df['Embarked'].map( {'S': 1, 'C': 2, 'Q': 3} )

In [1897]:
train_df.Embarked.value_counts()

1    646
2    168
3     77
Name: Embarked, dtype: int64

In [1898]:
test_df.Embarked.value_counts()

1    270
2    102
3     46
Name: Embarked, dtype: int64

In [1899]:
test_df.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked         int64
dtype: object

In [1900]:
# gender: replace categorial with numeric values

In [1901]:
train_df['Sex'] = train_df['Sex'].map( {'female': 1, 'male': 2} ).astype(int)

In [1902]:
test_df['Sex'] = test_df['Sex'].map( {'female': 1, 'male': 2} ).astype(int)

In [1903]:
train_df.Sex.value_counts()

2    577
1    314
Name: Sex, dtype: int64

In [1904]:
test_df.Sex.value_counts()

2    266
1    152
Name: Sex, dtype: int64

In [1905]:
train_df.describe()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,891.000000,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,1.647587,29.699118,0.523008,0.381594,32.204208,1.361392
std,257.353842,0.486592,0.836071,0.477990,14.526497,1.102743,0.806057,49.693429,0.635673
min,1.000000,0.000000,1.000000,1.000000,0.420000,0.000000,0.000000,0.000000,1.000000
25%,223.500000,0.000000,2.000000,1.000000,20.125000,0.000000,0.000000,7.910400,1.000000
50%,446.000000,0.000000,3.000000,2.000000,28.000000,0.000000,0.000000,14.454200,1.000000
75%,668.500000,1.000000,3.000000,2.000000,38.000000,1.000000,0.000000,31.000000,2.000000
max,891.000000,1.000000,3.000000,2.000000,80.000000,8.000000,6.000000,512.329200,3.000000


In [1906]:
test_df.describe()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,418.000000,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000,418.000000
mean,1100.500000,2.265550,1.636364,30.272590,0.447368,0.392344,35.627188,1.464115
std,120.810458,0.841838,0.481622,14.181209,0.896760,0.981429,55.907576,0.685516
min,892.000000,1.000000,1.000000,0.170000,0.000000,0.000000,0.000000,1.000000
25%,996.250000,1.000000,1.000000,21.000000,0.000000,0.000000,7.895800,1.000000
50%,1100.500000,3.000000,2.000000,27.000000,0.000000,0.000000,14.454200,1.000000
75%,1204.750000,3.000000,2.000000,39.000000,1.000000,0.000000,31.500000,2.000000
max,1309.000000,3.000000,2.000000,76.000000,8.000000,9.000000,512.329200,3.000000


In [1907]:
# replace missing values with mean of age column

In [1908]:
age_mean_tr = train_df['Age'].dropna().median()
print ('train mean age' , age_mean_tr)
age_mean_ts = test_df['Age'].dropna().median()
print ('test mean age' , age_mean_ts)

('train mean age', 28.0)
('test mean age', 27.0)


In [1909]:
fare_mean_ts = test_df['Fare'].dropna().median()

In [1910]:
train_df.loc[ (train_df.Age.isnull()), 'Age'] = age_mean_tr
test_df.loc[ (test_df.Age.isnull()), 'Age'] = age_mean_ts

In [1911]:
test_df.loc[ (test_df.Fare.isnull()), 'Fare'] = fare_mean_ts

In [1912]:
train_df['Fare'] = train_df['Fare'].astype(int)

In [1913]:
test_df['Fare'] = test_df['Fare'].astype(int)

In [1914]:
train_df.describe()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,1.647587,29.361582,0.523008,0.381594,31.785634,1.361392
std,257.353842,0.486592,0.836071,0.477990,13.019697,1.102743,0.806057,49.703730,0.635673
min,1.000000,0.000000,1.000000,1.000000,0.420000,0.000000,0.000000,0.000000,1.000000
25%,223.500000,0.000000,2.000000,1.000000,22.000000,0.000000,0.000000,7.000000,1.000000
50%,446.000000,0.000000,3.000000,2.000000,28.000000,0.000000,0.000000,14.000000,1.000000
75%,668.500000,1.000000,3.000000,2.000000,35.000000,1.000000,0.000000,31.000000,2.000000
max,891.000000,1.000000,3.000000,2.000000,80.000000,8.000000,6.000000,512.000000,3.000000


In [1915]:
test_df.describe()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,2.265550,1.636364,29.599282,0.447368,0.392344,35.133971,1.464115
std,120.810458,0.841838,0.481622,12.703770,0.896760,0.981429,55.855855,0.685516
min,892.000000,1.000000,1.000000,0.170000,0.000000,0.000000,0.000000,1.000000
25%,996.250000,1.000000,1.000000,23.000000,0.000000,0.000000,7.000000,1.000000
50%,1100.500000,3.000000,2.000000,27.000000,0.000000,0.000000,14.000000,1.000000
75%,1204.750000,3.000000,2.000000,35.750000,1.000000,0.000000,31.000000,2.000000
max,1309.000000,3.000000,2.000000,76.000000,8.000000,9.000000,512.000000,3.000000


In [1916]:
# delete all variables, we do not use in the model liek PassngerID, SibSp, ParchFare 
# also deleting fare, because we already use Pclass - which should be stong correlated to Fare

In [1917]:
train_df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex              int32
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare             int32
Cabin           object
Embarked         int64
dtype: object

In [1918]:
test_df.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex              int32
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare             int32
Cabin           object
Embarked         int64
dtype: object

In [1919]:
# save Passenger IDs for later reuse 
Pids = test_df['PassengerId'].values

In [1920]:
train_df['Age'] = train_df['Age'].astype(int)

In [1921]:
train_df.dtypes

PassengerId     int64
Survived        int64
Pclass          int64
Name           object
Sex             int32
Age             int32
SibSp           int64
Parch           int64
Ticket         object
Fare            int32
Cabin          object
Embarked        int64
dtype: object

In [1922]:
test_df['Age'] = test_df['Age'].astype(int)

In [1923]:
test_df.dtypes

PassengerId     int64
Pclass          int64
Name           object
Sex             int32
Age             int32
SibSp           int64
Parch           int64
Ticket         object
Fare            int32
Cabin          object
Embarked        int64
dtype: object

In [1924]:
test_df.dtypes

PassengerId     int64
Pclass          int64
Name           object
Sex             int32
Age             int32
SibSp           int64
Parch           int64
Ticket         object
Fare            int32
Cabin          object
Embarked        int64
dtype: object

In [1925]:
# Delete some variables, which as first sight seems to be not so influential
# columns = ['PassengerId', 'Name','Ticket','Cabin','Fare','SibSp','Parch']
columns = ['PassengerId','Name','Ticket','Cabin']

In [1926]:
train_df = train_df.drop(columns,axis=1)

In [1927]:
train_df.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,2,22,1,0,7,1
1,1,1,1,38,1,0,71,2
2,1,3,1,26,0,0,7,1
3,1,1,1,35,1,0,53,1
4,0,3,2,35,0,0,8,1


In [1928]:
test_df = test_df.drop(columns,axis=1)

In [1929]:
test_df.head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,2,34,0,0,7,3
1,3,1,47,1,0,7,1
2,2,2,62,0,0,9,3
3,3,2,27,0,0,8,1
4,3,1,22,1,1,12,1


In [1930]:
# now we have 4 variables and numeric values for the first random forest classification and transform them into an array
train_ar = train_df.values
test_ar = test_df.values

In [1931]:
len(train_ar)

891

In [1932]:
len(test_ar)

418

In [1933]:
surv_list = train_df['Survived'].values

In [1934]:
# use RandomForestClassifier for prediction and make instance RanFor
# number of trees = 100
rfc = RandomForestClassifier(n_estimators=100, min_samples_leaf=1)

In [1935]:
# use train_data for decision trees 
rfc = rfc.fit(train_ar[0::,1::], train_ar[0::,0] )

In [1936]:
train_del = np.delete(train_ar,1,axis=1)

In [1937]:
rfc.score(train_del, surv_list, sample_weight=None)

0.83613916947250277

In [1873]:
# use result decison trees of training set for the test set and prediction
Result = rfc.predict(test_ar).astype(int)

In [1874]:
Result

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0,

In [1875]:
Titanic_Submission = open("Titanic_Submission.csv", "wb")

In [1876]:
open_file_object = csv.writer(Titanic_Submission)
open_file_object.writerow(["PassengerId","Survived"])
open_file_object.writerows(zip(Pids, Result))
Titanic_Submission.close()